<a href="https://colab.research.google.com/github/alongutm/Harry-Potter-NLP/blob/main/Harry_Potter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !python -m spacy download en_core_web_lg  # Important! you need to restart runtime after install

!pip install turicreate
!pip install kaggle 
!pip install gensim
!pip install pyLDAvis
!pip install spaCy
!pip install afinn
!pip install nltk
!pip install plotly_express

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_lg')
book_file_path = "/content/Harry Potter and the Sorcerer's Stone.txt"
with open(book_file_path, 'r') as book_file:
  text = book_file.read()
text = text.replace("\n", " ")
doc = nlp(text)


OSError: ignored

In [ ]:
entities_dict= {}
for entity in doc.ents:
  label = entity.label_
  entity_lower = entity.text.lower().title()
  if  label not in entities_dict:
      entities_dict[label] = {entity_lower: 0}
  if entity_lower not in entities_dict[label]:
    entities_dict[label][entity_lower] = 1
  else:
    entities_dict[label][entity_lower] += 1

stopwords = set(STOPWORDS) | {'Yeh'}
entities_counts = {k: v for k, v in entities_dict["PERSON"].items() if v > 5 and k not in stopwords}
entites_counts_sorted = sorted(entities_counts.items(), key=lambda x: x[1], reverse=True)
print(entites_counts_sorted)  

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='black', 
                stopwords = stopwords, 
                min_font_size = 10,
                collocations=False)
wordcloud.generate_from_frequencies(frequencies=dict(entites_counts_sorted))
plt.figure(figsize = (20, 20), facecolor = None) 
plt.imshow(wordcloud)